This notebook is part of the various tests for scimap. It's intended to allow for visual evaluation of fits, etc.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
plt.xkcd()
import pandas as pd
import scimap
import numpy as np

## Prepare HDF File for Later Unit Tests


In [ ]:
scimap.import_aps_34IDE_map(
    directory="test-data-xrd/xrd-map-34-ID-E/",
    wavelength=0.4959,
    shape=scimap.shape(rows=13, columns=11),
    step_size=scimap.default_units.um(1),                      
    beamstop=0.49,                            
    hdf_filename="test-data-xrd/xrd-map-34-ID-E.hdf"
);

## Peak Fitting

In [6]:
celref_peaks = [
            ('012', 3.4746228816945104, 25.637288649553085),
            ('104', 2.5479680737754244, 35.22223164557721),
            ('110', 2.375, 37.88141047624646),
            ('006', 2.1636666666666664, 41.74546075011751),
            ('113', 2.0820345582756135, 43.46365474219995),
            ('024', 1.7373114408472552, 52.68443192186963),
            ('116', 1.5994489779586798, 57.62940019834231),
]
two_theta = np.array([p[2] for p in celref_peaks])
print(two_theta)
print(scimap.tubes['Cu'].kalpha)
scimap.twotheta_to_q(two_theta, scimap.tubes['Cu'].kalpha)

[ 25.63728865  35.22223165  37.88141048  41.74546075  43.46365474
  52.68443192  57.6294002 ]
1.5418666666666667


[autoreload of xrd.phase failed: Traceback (most recent call last):
  File "/usr/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: 'NoneType' object has no attribute 'unit_cell'
]


array([ 1.80822894,  2.46585258,  2.64543732,  2.90382698,  3.0176798 ,
        3.61645787,  3.92817384])